In [33]:
import os
import datetime
import xlrd
import dataset

In [31]:
def dated_file_name(extension, prefix):
    filename = prefix + "_" + datetime.datetime.now().isoformat() + "." + extension
    return filename

In [25]:
data_dir = "/Users/jjpr/dev/scratch/mkgu_scratch/data"

In [55]:
class MappingFile(object):
    def __repr__(self):
        return self.__class__.__name__ + str(vars(self))

In [38]:
mf_paths = [os.path.join(data_dir, x) for x in os.listdir(data_dir) if x.endswith(".xls")]
mf_paths

['/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-4F.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-5E.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-5F.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-5H.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-8E.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/2132-5-SN0656.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/2132-8-SN0657.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/2466-1-SN0658.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/2467-6_SN0649.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/5602-2 SN1025-000936.xls']

In [70]:
def is_comment_row(row):
    return str(row[0].value).lstrip().startswith("//")

In [81]:
def get_mapping(i, row):
    return {
        "column": row[0].value, 
        "row": row[1].value, 
        "bank_name": row[2].value,
        "electrode_number_in_bank": row[3].value,
        "electrode_label": row[4].value,
        "index_in_mapping_sheet": i
    }

In [89]:
def get_array_id(row):
    stripped = row[0].value.strip()
    chopped = stripped[len("Cerebus mapping for array "):]
    squoze = ''.join(chopped.split())
    return squoze

'2467-6/SN0649'

In [90]:
def get_mapping_file(path):
    mf = MappingFile()
    mf.xls_path = path
    mf.workbook = xlrd.open_workbook(path)
    mf.mapping_worksheet = mf.workbook.sheet_by_name("Cerebus mapping")
    mf.comment_lines = set()
    mf.description_row = None
    mf.mappings = []
    for i, row in enumerate(mf.mapping_worksheet.get_rows()):
        if is_comment_row(row):
            mf.comment_lines.add(i)
        elif mf.description_row is None:
            mf.description_row = row
        else:
            mf.mappings.append(get_mapping(i, row))
    mf.array_id = get_array_id(mf.description_row)
    return mf

In [91]:
mfs = []

for path in mf_paths:
    mf = get_mapping_file(path)
    mfs.append(mf)

mfs

[MappingFile{'xls_path': '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-4F.xls', 'workbook': <xlrd.book.Book object at 0x105396048>, 'mapping_worksheet': <xlrd.sheet.Sheet object at 0x1053e9518>, 'comment_lines': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}, 'description_row': [text:'Cerebus mapping for array 090609B-4F/SN0559', empty:'', empty:'', empty:'', empty:'', empty:'', text:'elec', empty:''], 'mappings': [{'column': 8.0, 'row': 9.0, 'bank_name': 'A', 'electrode_number_in_bank': '14', 'electrode_label': 'elec99', 'index_in_mapping_sheet': 13}, {'column': 7.0, 'row': 9.0, 'bank_name': 'A', 'electrode_number_in_bank': '10', 'electrode_label': 'elec98', 'index_in_mapping_sheet': 14}, {'column': 6.0, 'row': 9.0, 'bank_name': 'A', 'electrode_number_in_bank': '8', 'electrode_label': 'elec97', 'index_in_mapping_sheet': 15}, {'column': 5.0, 'row': 9.0, 'bank_name': 'A', 'electrode_number_in_bank': '6', 'electrode_label': 'elec96', 'index_in_mapping_sheet': 16}, {'column': 4.0, 'row': 9.0

In [93]:
[x.array_id for x in mfs]

['090609B-4F/SN0559',
 '090609B-5E/SN00547',
 '090609B-5F/SN0548',
 '090609B-5H/SN0549',
 '090609B-8E/SN0552',
 '2132-5/SN0656',
 '2132-8/SN0657',
 '2466-1/SN0658',
 '2467-6/SN0649',
 '5602-2/SN1025-000936']

In [102]:
try: 
    db
except NameError:
    db_file = os.path.join(data_dir, dated_file_name("db", "array_meta"))
    db_url = "sqlite:///" + db_file
    db = dataset.connect(db_url)

In [103]:
db

<Database(sqlite:////Users/jjpr/dev/scratch/mkgu_scratch/data/array_meta_2018-01-31T10:53:03.275843.db)>

In [104]:
db["foo"].insert({"bar": "baz"})

1

In [96]:
tbl_array = db["array"]
tbl_mapping = db["mapping"]
for mf in mfs:
    array_dict = {"blackrock_id": mf.array_id, "mapping_xls":mf.xls_path}
    array_id = tbl_array.upsert(array_dict, keys=["blackrock_id"])
    for mapping in mf.mappings:
        insert = dict(mapping)
        insert["array_id"] = array_id
        tbl_mapping.insert(insert)

OperationalError: (sqlite3.OperationalError) unable to open database file (Background on this error at: http://sqlalche.me/e/e3q8)